In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
import geopy.distance



In [8]:
conflict_df = pd.read_csv('../data/conflict_data_irq.csv')

In [9]:
conflict_df.drop(0, inplace=True)

In [10]:
conflict_df.columns

Index(['id', 'year', 'active_year', 'start_year', 'end_year',
       'type_of_violence', 'conflict_new_id', 'conflict_name', 'dyad_new_id',
       'dyad_name', 'side_a_new_id', 'gwnoa', 'side_a', 'side_b_new_id',
       'gwnob', 'side_b', 'number_of_sources', 'source_article',
       'source_office', 'source_date', 'source_headline', 'source_original',
       'where_prec', 'where_coordinates', 'adm_1', 'adm_2', 'latitude',
       'longitude', 'geom_wkt', 'priogrid_gid', 'country', 'country_id',
       'iso3', 'region', 'event_clarity', 'date_prec', 'date_start',
       'date_end', 'deaths_a', 'deaths_b', 'deaths_civilians',
       'deaths_unknown', 'low', 'best', 'high'],
      dtype='object')

In [11]:
conflict_df.replace({'Al Anbār province':'Anbar',
                            'Nīnawá province':'Ninewa',
                           'Baghdād province':'Baghdad', 
                            'Dahūk province':'Dahuk', 
                          'Diyālá province':'Diyala',
                           'Kirkūk province':'Kirkuk',
                           'Şalāḩ ad Dīn province':'Salah al-Din'}, inplace=True)

In [12]:
gov_names = ['Anbar', 'Ninewa', 'Baghdad', 'Dahuk', 'Diyala', 'Salah al-Din']

In [ ]:
conflict

In [6]:
conflict_df = conflict_df[conflict_df['adm_1'].isin(gov_names)]


In [7]:
conflict_df['adm_1'].unique()

array([], dtype=object)

In [ ]:
conflict_df['adm_1'].unique()

In [ ]:
returnee_filepaths = [f for f in listdir("../data/returnees/") if f.endswith('.xlsx')]


In [ ]:

# master_filepaths = [s for s in listdir("../data/master_list/") if s.endswith('.xlsx')]
# mas_df = pd.concat((pd.read_excel("../data/master_list/"+s).assign(date=s[31:43]) 
#                    for s in master_filepaths), ignore_index=True) 
#mas_df['date'] = pd.to_datetime(mas_df['date'])

In [ ]:
ret_df = pd.concat((pd.read_excel("../data/returnees/"+f).assign(date=f[35:47])
                  for f in returnee_filepaths), ignore_index=True)

In [ ]:
ret_df['District'].unique()

In [ ]:
ret_df['Governorate'].unique()

In [ ]:
ret_df.dropna(how='all', inplace=True)

In [ ]:
ret_df['date'] = pd.to_datetime(ret_df['date'])

In [ ]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)



In [ ]:
ret_df = trim_all_columns(ret_df)

In [ ]:
ret_df.columns

In [ ]:

ret_df.sort_values(['Location_name', 'date'], inplace=True)




In [ ]:
ret_df.rename(columns={'Pre June14 Period of displacement':'disp_preJun14',
                      'June July14 Period of displacement':'disp_JunJuly14',
                      'August14 Period of displacement':'disp_Aug14',
                      'Post September 14 Period of displacement':'disp_postSep14',
                      'Post April15 Period of displacement':'disp_postApr15',
                      'Post March 16 Period of displacement':'disp_postMar16',
                      'Post 17 October 16 Period of displacement': 'disp_post17Oct16',
                      'July 17 Period of displacement':'disp_Jul17',
                      'Jan19':'disp_Jan19'}, inplace=True)

In [ ]:
ret_df

In [ ]:
ret_df.dropna(how='any', inplace=True)

In [ ]:
ret_df['ret_delta'] = ret_df.groupby(['Location_name'])['Returnee Families'].transform(lambda x: x.diff()) 



In [ ]:
ret_df.sort_values(['Location_name', 'date'], inplace=True)

In [ ]:
ret_df.reset_index(inplace=True)

In [ ]:
#ret_df.remove(column='index')

In [ ]:
ret_df

In [ ]:
ret_df[ret_df['Location_name'] == 'Gwer']

In [ ]:
ret_df.at[14022, 'ret_delta']

In [ ]:
#should be functionized 
for i in range(len(ret_df)):
    if np.isnan(ret_df.at[i, 'ret_delta']):
        ret_df.at[i, 'ret_delta'] = ret_df.at[i, 'Returnee Families']
    else:
        None

In [ ]:
ret_df[ret_df['Location_name'] == 'Gwer']

In [ ]:
conflict_df.columns

In [ ]:
conflict_df['date_start'] = pd.to_datetime(conflict_df['date_start'])

In [ ]:
conflict_df['date_end'] = pd.to_datetime(conflict_df['date_start'])

In [ ]:
conflict_df.columns

In [ ]:
conflict_df['adm_2'].unique()


In [ ]:
ser = pd.Series(index=ret_df.index, data=0)

In [ ]:
ret_df['District'].unique()

In [ ]:
print(7473*26000)

In [ ]:
for ix, coord in enumerate(zip(conflict_df['latitude'], conflict_df['longitude'])):
    for idx, rcoord in enumerate(zip(ret_df['Latitude'], ret_df['Longitude'])):
        if geopy.distance.distance(coord, rcoord).km > 10:
            print(idx)
            ser[idx] += 1
        else:
            print(idx)